In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipywidgets as widgets
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [58]:
turmas_url = 'http://bit.do/turmas_ufrn_2017_2'
notas_url = 'http://bit.do/notas_ufrn_2017_2'
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'
docentes_url = 'http://bit.do/docentes_ufrn_2018'

turmas = pd.read_csv(turmas_url, sep=';')
notas = pd.read_csv(notas_url, sep=';')
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])
docentes = pd.read_csv(docentes_url, sep=';')

turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas.dropna(inplace=True)
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57602957,5753061.0,ART2077,LATO SENSU,PRODUÇÃO COREOGRÁFICA I,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T23456
2,57602962,5752714.0,ART2083,LATO SENSU,VIESES ARTÍSTICOS CONTEMPORÂNEOS EM DANÇA ...,DEPARTAMENTO DE ARTES,ABERTA,40.0,1M345 7M2345 17T23456
3,57602963,3031948.0,ART2086,LATO SENSU,PRÁTICAS DE IMPROVISAÇÃO PARA A CENA DAS DANÇA...,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T3456
4,57603192,5752917.0,MUT491,TÉCNICO,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,20.0,3T34
5,57603193,35466.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T456


In [59]:
# filtrando as turmas com código IMD
turmas_imd = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd = turmas_imd[turmas_imd.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd.capacidade_aluno = turmas_imd.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd.id_servidor = turmas_imd.id_servidor.astype(np.int64)
turmas_imd.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
9195,57608376,5755939,IMD0017,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,5N34 (24/07/2017 - 16/12/2017)
9196,57608377,25889,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,65,246M3456 (24/07/2017 - 16/12/2017)
9197,57608377,5756386,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,65,246M3456 (24/07/2017 - 16/12/2017)
9198,57608377,5759424,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,65,246M3456 (24/07/2017 - 16/12/2017)
9199,57608378,5757969,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,67,246N1234 (24/07/2017 - 16/12/2017)


In [66]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof = pd.merge(turmas_imd, docentes, on='id_servidor')[colunas]
turmas_imd_nomesprof.head()
lista_prof = turmas_imd_nomesprof.nome_y.unique()

In [67]:
#Essa celula pega a avaliação dos professores do imd
avaliacao = pd.read_csv("avaliacaodocencia.csv", sep=';')
av_imd = avaliacao[avaliacao['nome_docente'].isin(lista_prof)]
av_imd.head()
#avaliacao2 = (avaliacao[(avaliacao.ano.astype(str).str.contains("2016")) | (avaliacao.ano.astype(str).str.contains("2017"))]

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP
7787,22764,JAIR CAVALCANTI LEITE,1233659,2014,2,17,9.09,1.57,9.00,1.60,8.84,1.97
7788,22764,JAIR CAVALCANTI LEITE,1233661,2014,2,36,9.48,1.26,9.43,1.20,8.22,2.23
7789,22764,JAIR CAVALCANTI LEITE,1242644,2015,1,34,9.41,1.04,8.86,1.73,8.14,2.32
7790,22764,JAIR CAVALCANTI LEITE,57566712,2015,2,36,9.81,0.51,9.48,1.35,8.55,2.15
7791,22764,JAIR CAVALCANTI LEITE,57566818,2015,2,14,9.69,0.48,8.13,2.44,7.87,2.69


In [99]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

def evolucao(x):
    dados_prof = avaliacao.loc[avaliacao['nome_docente'] == x]
    return (dados_prof)

interact(evolucao, x=lista_prof,);

A Jupyter Widget